In [1]:
import pandas as pd
import pickle
import re
from easynmt import EasyNMT
from pandarallel import pandarallel
import tensorflow as tf
import torch
model = EasyNMT('opus-mt')

In [2]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print(f'Found GPU at: {device_name}')
    
if torch.has_mps:    
    device = torch.device("mps")
else:
    print('using the CPU')
    device = torch.device("cpu")

Metal device set to: Apple M1
Found GPU at: /device:GPU:0


2023-04-11 19:57:45.794217: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-11 19:57:45.794790: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
english_data = pd.read_csv('processed_datasets/combined_english_tweets.csv')
english_data['processed_tweet'] = english_data['processed_tweet'].apply(lambda x: re.sub(r'[\.]', '', str(x))) \
                                    .apply(lambda x: re.sub(r'\s\s',' ', str(x)))
english_sentences = english_data['processed_tweet'].tolist()

In [28]:
french_translations = model.translate(english_sentences, source_lang = 'en', target_lang = 'fr')

In [31]:
pickle.dump(french_translations, open('processed_datasets/french_translations.pkl', 'wb'))

In [33]:
german_translations = model.translate(english_sentences, source_lang = 'en', target_lang = 'de')
pickle.dump(german_translations, open('processed_datasets/german_translations.pkl', 'wb'))

In [34]:
italian_translations = model.translate(english_sentences, source_lang = 'en', target_lang = 'it')
pickle.dump(italian_translations, open('processed_datasets/italian_translations.pkl', 'wb'))

Downloading:   0%|          | 0.00/771k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/795k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/327M [00:00<?, ?B/s]

In [18]:
french_translations = pickle.load(open('processed_datasets/french_translations.pkl', 'rb'))
german_translations = pickle.load(open('processed_datasets/german_translations.pkl', 'rb'))
italian_translations = pickle.load(open('processed_datasets/italian_translations.pkl', 'rb'))

english_labels = french_labels = german_labels = italian_labels = english_data['stance'].tolist()

train_labels = english_labels + french_labels + german_labels + italian_labels 
train_text = english_sentences + french_translations + german_translations + italian_translations

backtranslated_train_data = pd.DataFrame({'processed_tweet': train_text, 'stance': train_labels})
backtranslated_train_data.to_csv('processed_datasets/backtranslated_train_data.csv')

In [6]:
print(model.translate('This is a sentence we want to translate to German', target_lang='fr'))

#Translate several sentences to German
sentences = ['You can define a list with sentences.',
             'All sentences are translated to your target language.',
             'Note, you could also mix the languages of the sentences.']
print(model.translate(sentences, target_lang='fr'))

C'est une phrase que nous voulons traduire en allemand
['Vous pouvez définir une liste avec des phrases.', 'Toutes les phrases sont traduites dans votre langue cible.', 'Notez que vous pouvez également mélanger les langues des phrases.']


In [9]:
langs = english_data['processed_tweet'].apply(lambda x: model.language_detection(str(x)))

In [10]:
print(langs)

0        en
1        en
2        en
3        en
4        en
       ... 
4488     en
4489     en
4490    nds
4491     en
4492     en
Name: processed_tweet, Length: 4493, dtype: object


In [20]:
for ix, lang in langs.iteritems():
    if lang!='en':
        print(lang, ix)

de 192
hu 395
hu 488
de 779
hu 1170
tl 1249
de 1916
de 1969
tl 2047
tr 2055
tr 2061
ht 2067
ht 2074
de 2110
it 3115
es 3118
es 3486
it 3717
nds 4490


/var/folders/ch/czk31wg97tvdq3v6ky8dnt5r0000gn/T/ipykernel_7267/2961108887.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for ix, lang in langs.iteritems():


In [26]:
english_data.iloc[395]

Unnamed: 0                                                       395
id                                               1368160808853667842
tweet              Vaccine Ready!\n#AstraZeneca \nVinaka @MOHFiji...
stance                                                      positive
processed_tweet              Vaccine Ready! AstraZeneca Vinaka &amp;
Name: 395, dtype: object

In [27]:
model.translate(english_data.iloc[395]['processed_tweet'], source_lang = 'en',target_lang='fr')

'Prêt pour le vaccin! AstraZeneca Vinaka &amp;'

In [23]:
model.language_detection(english_data.iloc[192]['processed_tweet'])

'de'